# Positive/negativeコメントを品詞で見る

Date: 2023/12/3

In [7]:
import spacy

!python3 -m spacy download "en_core_web_sm"
!python3 -m spacy download "ja_core_news_sm"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 9.6 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')


In [2]:
import yaml

with open('../data/impressions.yaml', 'r') as f:
    imp = yaml.safe_load(f.read())

In [3]:
nlp = spacy.load('ja_core_news_sm')
imp.keys()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['negative', 'positive', 'positive_negative', 'random'])

In [4]:
imp_ = []
for doc in nlp.pipe(imp['positive_negative']):
    parts = []
    prev = 0
    imp__ = []
    for idx, token in enumerate(doc):
        #print(f'{token.text}[{idx}]({token.pos_}) ', end='')
        if token.pos_ == 'SCONJ' and token.text in ('が'):
            parts.append([prev, idx+1])
            prev = idx+1
    for p0, p1 in parts:
        #print(doc[p0:p1].text.strip('、'), end=': ')
        imp__.append(doc[p0:p1-1].text.strip('、'))
    if len(parts) > 0:
        last_p = parts[-1][1]
        if last_p < len(doc):
            #print(doc[last_p: len(doc)].text.strip('、'))
            imp__.append(doc[last_p: len(doc)].text.strip('、'))
        else:
            print()
    imp_.append(imp__)
print(imp_)

[['ショールームの清潔さには好感が持てました', '一部の展示品が傷んでいるのが気になりました。'], ['スタッフの丁寧な対応がありました', '商品の説明が少なく、理解しにくかったです。'], ['展示品の品質は高く、特にデザインに魅了されました', '価格が高すぎると感じました。'], ['ショールームの広さとレイアウトは素晴らしかった', '一部のエリアが混雑していました。'], [], ['インテリアデザインがモダンで印象的でした', '一部のエリアが古く感じられました。'], ['スタッフの対応は親切で好感が持てました', '混雑しておりゆっくり見学できませんでした。'], ['展示品のディスプレイは美しく、高い品質が感じられました', '価格が高すぎると感じました。'], ['ショールーム内の照明が良好で、商品が見やすかった', '一部のエリアが薄暗かったです。'], ['商品の価格設定が適切で良心的でした', 'スタッフの知識が不足しているように感じました。'], ['展示品の配置が工夫され、テーマ性がありました', '一部の商品が見つけにくかったです。'], ['ショールーム内の案内が分かりやすく、スタッフの対応も良かった', '混雑が気になりました。'], ['商品の新しさとデザインに魅了されました', 'スタッフの態度が冷淡で残念でした。'], ['ショールームの雰囲気が明るく、居心地が良かった', '一部の商品が陳腐に感じられました。'], ['展示品の保守状態が良好で、商品の信頼性に安心感を覚えました', '価格が心配です。'], ['ショールームの配置が効果的で見やすかった', 'スタッフの忙しさが対応に影響していました。'], ['スタッフの知識が豊富で的確でした', '商品の新しさに欠け、刷新が必要かもしれません。'], ['商品の魅力が引き立つディスプレイでした', 'スタッフの説明が簡略化されていた印象です。'], ['ショールーム内の音楽が心地よく、雰囲気が良かった', '一部エリアが騒がしかったです。'], ['展示品のデザインが洗練されており、一部の商品には本当に惹かれました', '価格が心配です。'], ['ショールームの温度が快適で、滞在が楽しかった', '一部のエリアが寒かったです。'], ['商品の新しさが感じられ

In [5]:
# 色々とモデルを試した方が良い。

from transformers import pipeline
sentiment1 = pipeline(model='christian-phu/bert-finetuned-japanese-sentiment')
sentiment2 = pipeline(model='llm-book/bert-base-japanese-v3-marc_ja')

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
# llm-book/bert-base-japanese-v3-marc_ja の方がきちんと negative 判定してくれる。
for i in imp_:
    print(i)
    print([s['label'] for s in sentiment1(i)])
    print([s['label'] for s in sentiment2(i)])
    print()

['ショールームの清潔さには好感が持てました', '一部の展示品が傷んでいるのが気になりました。']
['positive', 'neutral']
['positive', 'negative']

['スタッフの丁寧な対応がありました', '商品の説明が少なく、理解しにくかったです。']
['positive', 'neutral']
['positive', 'negative']

['展示品の品質は高く、特にデザインに魅了されました', '価格が高すぎると感じました。']
['positive', 'neutral']
['positive', 'negative']

['ショールームの広さとレイアウトは素晴らしかった', '一部のエリアが混雑していました。']
['positive', 'negative']
['positive', 'negative']

[]
[]
[]

['インテリアデザインがモダンで印象的でした', '一部のエリアが古く感じられました。']
['positive', 'neutral']
['positive', 'negative']

['スタッフの対応は親切で好感が持てました', '混雑しておりゆっくり見学できませんでした。']
['positive', 'negative']
['positive', 'negative']

['展示品のディスプレイは美しく、高い品質が感じられました', '価格が高すぎると感じました。']
['positive', 'neutral']
['positive', 'negative']

['ショールーム内の照明が良好で、商品が見やすかった', '一部のエリアが薄暗かったです。']
['positive', 'neutral']
['positive', 'negative']

['商品の価格設定が適切で良心的でした', 'スタッフの知識が不足しているように感じました。']
['positive', 'neutral']
['positive', 'negative']

['展示品の配置が工夫され、テーマ性がありました', '一部の商品が見つけにくかったです。']
['neutral', 'neutral']
['positive', 'negative']

['ショールー

## Enlgish 

In [9]:
with open('../data/impressions_en.yaml', 'r') as f:
    imp = yaml.safe_load(f.read())

dict_keys(['negative', 'positive', 'positive_negative', 'random'])

In [10]:
nlp = spacy.load('en_core_web_sm')
imp.keys()

dict_keys(['negative', 'positive', 'positive_negative', 'random'])

In [18]:
imp_ = []
for doc in nlp.pipe(imp['positive_negative']):
    '''
    for token in doc:
        print(f'{token.text}({token.pos_}) ', end='')
    print()
    '''
    for sent in doc.sents:
        print(sent.text.split(','))

['The showroom was absolutely stunning!']
['The modern designs and vibrant colors were impressive.']
['However', ' the lack of natural lighting made some areas feel a bit dim.']
['I was amazed by the innovative features in the showroom.']
['The interactive displays were engaging', ' but the crowded layout made navigation a bit challenging.']
["The showroom's minimalist approach was refreshing."]
['The sleek designs and clean lines were appealing.']
['However', ' the limited seating options were a drawback.']
['I loved the variety of products in the showroom.']
['The knowledgeable staff provided great insights.']
['On the downside', ' the high prices were a bit discouraging.']
["The showroom's unique theme caught my attention."]
['The creative displays were captivating', ' but the loud background music made it hard to focus on product details.']
["The showroom's spacious layout allowed for easy exploration."]
['The friendly staff added to the positive experience.']
['However', ' the out